# Modelo Xgboost 1 

In [1]:
import common.common_machine_learning as common
import common.metrica as metrica
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import  KFold
import xgboost as xgb
import time

### Ignorar Future Warnings 

In [4]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Constantes 

In [5]:
TARGET = 'precio'
TEST_FRACCION = 0.25
RANDOM_SEMILLA = 1
XGB_OBJECTIVE = 'reg:squarederror'
K = 3

In [6]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

In [7]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

## Set de datos 

In [8]:
train = common.cargar_set_optimizado('sets_de_datos/train.csv')

 Descartamos las siguientes columnas para este modelo inicial:

- titulo, descripcion, direccion : strings complejas que requieren simplificacion previa a encodign.
- lat, lng : valores numericos que cuyos nulls no se pueden inputar por valores con sentido de forma simple
- idzona, fecha : mismo motivos que lat, lng

In [9]:
train = common.eliminar_columnas_complejas(train)

In [10]:
X = train.drop([TARGET], axis = 1).copy()
y = train[TARGET].copy()

In [15]:
X.shape

(240000, 15)

In [16]:
X.head()

,id,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,254099,0,93,8,NaN,2.0,1.0,2.0,80.0,80.0,False,False,False,False,False
1,53461,3,370,8,10.0,3.0,2.0,2.0,268.0,180.0,False,False,False,True,True
2,247984,2,768,14,5.0,3.0,2.0,2.0,144.0,166.0,False,False,False,False,False
3,209067,2,862,10,1.0,2.0,1.0,1.0,63.0,67.0,False,False,False,True,True
4,185997,0,852,14,10.0,2.0,1.0,1.0,95.0,95.0,False,False,False,False,False


In [17]:
y.head()

0    2273000.0
1    3600000.0
2    1200000.0
3     650000.0
4    1150000.0
Name: precio, dtype: float32

In [12]:
X = common.label_encode_strings_simples(X)

## ¿Cuanto tarda un K-fold cross-validation?

No puedo usar RMSLE a la ligera, pues XGBoost puede devolver valores negativos dada su prediccion.  
Cuando empieze a probar distintos hiperparametros deberé tener esto en cuenta.  
Usare mean square error para esta prueba,

In [23]:
kfold = KFold(n_splits=K, shuffle=True, random_state=RANDOM_SEMILLA)

scores = []

inicio = time.time()
for train_index, test_index in kfold.split(X):   
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    xgb_model = xgb.XGBRegressor(objective="reg:squarederror")
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))
fin = time.time()
minutos = (fin-inicio)/60
print('Tiempo que tarda K-Fold CV para K elegido: {0:.1f} minutos'.format(minutos))
print()
display_scores(np.sqrt(scores))

Tiempo que tarda K-Fold CV para K elegido: 0.4 minutos

Scores: [1207357.9 1209633.1 1213039.4]
Mean: 1210010.125
Std: 2334.732
